In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.interpolate import griddata

In [2]:
# general mesh info
file_14 = r'./fort.14'
mesh_info = []
with open(file_14, 'r', encoding='utf-8') as f:
    for i in range(0,2):
        wz = f.readline()
        wz = wz.strip('\n')
        wz = wz.split(' ')
        print(wz)
        mesh_info.append(wz)
f.close()
mesh_info
ncell = np.int64(mesh_info[1][0])
npoint = np.int64(mesh_info[1][1])
print(ncell, npoint)

['OceanMesh2D', '']
['24235', '13375', '']
24235 13375


In [3]:
points = pd.read_csv(file_14, skiprows=2, header=None, nrows=npoint, sep='\s+')

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\byy\AppData\Local\Temp\ipykernel_9716\908926587.py:1: SyntaxWarning: invalid escape sequence '\s'
  points = pd.read_csv(file_14, skiprows=2, header=None, nrows=npoint, sep='\s+')


In [4]:
points.columns = ['id', 'lon', 'lat', 'depth']

## Create an idealized `fort.22` input file (an idealized case)

In [6]:
grid_out = np.vstack((points['lon'], points['lat'])).T

In [7]:
grid_out

array([[-77.59189357,  44.06330555],
       [-77.59503948,  44.06824093],
       [-77.58567537,  44.06187312],
       ...,
       [-79.79614082,  43.3220185 ],
       [-79.80016943,  43.31307469],
       [-79.79956704,  43.31790015]])

In [17]:
import os
os.system("rm ./fort.22")
save_file = r'./fort.22'

date_ran = pd.date_range("2019-10-30 00", "2019-11-03 00", freq="1h")
time_count = len(date_ran)
u10_out = 8.0*np.ones((npoint))
v10_out = 0.0*np.ones((npoint))
sp_out = 95000.0*np.ones((npoint))/9800
outputs =  np.empty((0,4))
for itime in range(time_count):
    output = np.vstack((points['id'],u10_out,v10_out,sp_out)).T
    outputs = np.concatenate([outputs, output])
    print(str(itime)+': '+str(date_ran[itime]))

save_file = r'./fort.22'
np.savetxt(save_file, outputs, fmt='%i %1.4f %1.4f %1.4f')

0: 2019-10-30 00:00:00
1: 2019-10-30 01:00:00
2: 2019-10-30 02:00:00
3: 2019-10-30 03:00:00
4: 2019-10-30 04:00:00
5: 2019-10-30 05:00:00
6: 2019-10-30 06:00:00
7: 2019-10-30 07:00:00
8: 2019-10-30 08:00:00
9: 2019-10-30 09:00:00
10: 2019-10-30 10:00:00
11: 2019-10-30 11:00:00
12: 2019-10-30 12:00:00
13: 2019-10-30 13:00:00
14: 2019-10-30 14:00:00
15: 2019-10-30 15:00:00
16: 2019-10-30 16:00:00
17: 2019-10-30 17:00:00
18: 2019-10-30 18:00:00
19: 2019-10-30 19:00:00
20: 2019-10-30 20:00:00
21: 2019-10-30 21:00:00
22: 2019-10-30 22:00:00
23: 2019-10-30 23:00:00
24: 2019-10-31 00:00:00
25: 2019-10-31 01:00:00
26: 2019-10-31 02:00:00
27: 2019-10-31 03:00:00
28: 2019-10-31 04:00:00
29: 2019-10-31 05:00:00
30: 2019-10-31 06:00:00
31: 2019-10-31 07:00:00
32: 2019-10-31 08:00:00
33: 2019-10-31 09:00:00
34: 2019-10-31 10:00:00
35: 2019-10-31 11:00:00
36: 2019-10-31 12:00:00
37: 2019-10-31 13:00:00
38: 2019-10-31 14:00:00
39: 2019-10-31 15:00:00
40: 2019-10-31 16:00:00
41: 2019-10-31 17:00:00
42

## Create the `fort.22` file based on ERA5 data (`NWS=5`)

In [5]:
grid_out = np.vstack((points['lon'], points['lat'])).T

In [6]:
ds = xr.open_dataset(r'./out.nc')
ds

<xarray.Dataset>
Dimensions:     (valid_time: 98, longitude: 51, latitude: 26)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 2019-10-29T23:00:00 ... 2019-11-03
  * longitude   (longitude) float64 -80.5 -80.4 -80.3 ... -75.7 -75.6 -75.5
  * latitude    (latitude) float64 45.0 44.9 44.8 44.7 ... 42.8 42.7 42.6 42.5
Data variables:
    u10         (valid_time, latitude, longitude) float32 ...
    v10         (valid_time, latitude, longitude) float32 ...
    sp          (valid_time, latitude, longitude) float32 ...
Attributes:
    CDI:                     Climate Data Interface version 2.0.4 (https://mp...
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    history:                 Sun Jan 05 16:20:48 2025: cdo mergetime Oct-29.n...
    CDO:                     Climate Data Operators version 2.0.4 (https://mp...

In [7]:
lon = ds.longitude
lat = ds.latitude
lons,lats = np.meshgrid(lon, lat)
date_ran = pd.date_range("2019-10-30 00", "2019-11-03 00", freq="1h")
time_count = len(date_ran)
grid_in = np.concatenate([lons.reshape(-1,1), lats.reshape(-1,1)], axis=1)

In [8]:
import os
os.system("rm ./fort.22")
save_file = r'./fort.22'

outputs =  np.empty((0,4))
for itime in range(time_count):
    u10_in = ds.u10.sel(valid_time=date_ran[itime], method='nearest')
    v10_in = ds.v10.sel(valid_time=date_ran[itime], method='nearest')
    sp_in = ds.sp.sel(valid_time=date_ran[itime], method='nearest')
    u10_out = griddata(grid_in, np.array(u10_in).flatten(), grid_out, method='linear')
    v10_out = griddata(grid_in, np.array(v10_in).flatten(), grid_out, method='linear')
    sp_out = griddata(grid_in, np.array(sp_in).flatten(), grid_out, method='linear')/9800.0
    output = np.vstack((points['id'],u10_out,v10_out,sp_out)).T
    outputs = np.concatenate([outputs, output])
    print(str(itime)+': '+str(date_ran[itime]))

save_file = r'./fort.22'
np.savetxt(save_file, outputs, fmt='%i %1.4f %1.4f %1.4f')

0: 2019-10-30 00:00:00
1: 2019-10-30 01:00:00
2: 2019-10-30 02:00:00
3: 2019-10-30 03:00:00
4: 2019-10-30 04:00:00
5: 2019-10-30 05:00:00
6: 2019-10-30 06:00:00
7: 2019-10-30 07:00:00
8: 2019-10-30 08:00:00
9: 2019-10-30 09:00:00
10: 2019-10-30 10:00:00
11: 2019-10-30 11:00:00
12: 2019-10-30 12:00:00
13: 2019-10-30 13:00:00
14: 2019-10-30 14:00:00
15: 2019-10-30 15:00:00
16: 2019-10-30 16:00:00
17: 2019-10-30 17:00:00
18: 2019-10-30 18:00:00
19: 2019-10-30 19:00:00
20: 2019-10-30 20:00:00
21: 2019-10-30 21:00:00
22: 2019-10-30 22:00:00
23: 2019-10-30 23:00:00
24: 2019-10-31 00:00:00
25: 2019-10-31 01:00:00
26: 2019-10-31 02:00:00
27: 2019-10-31 03:00:00
28: 2019-10-31 04:00:00
29: 2019-10-31 05:00:00
30: 2019-10-31 06:00:00
31: 2019-10-31 07:00:00
32: 2019-10-31 08:00:00
33: 2019-10-31 09:00:00
34: 2019-10-31 10:00:00
35: 2019-10-31 11:00:00
36: 2019-10-31 12:00:00
37: 2019-10-31 13:00:00
38: 2019-10-31 14:00:00
39: 2019-10-31 15:00:00
40: 2019-10-31 16:00:00
41: 2019-10-31 17:00:00
42

In [9]:
len(date_ran)

97